# Check if smaller LMs splite the manthematical definitions and titles intro smaller wordpieces

In [1]:
import transformers
import pandas as pd
import logging
import torch
import numpy as np

from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModel
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

transformers.logging.set_verbosity_error()

LOG_FORMAT = '%(asctime)s : %(filename)s : %(funcName)s : %(levelname)s : %(message)s'
logging.basicConfig(filename='SentencePairCLS.log', level=logging.INFO, format=LOG_FORMAT)
logger = logging.getLogger("SentencePairCLS")

cuda


In [2]:
import pickle
def pkl_vars(varname, filename):
    with open(filename, 'wb') as file:
        pickle.dump(varname, file)
        
def reload_vars(filename):
    this_var = None
    with open(filename, 'rb') as file:
        this_var = pickle.load(file)      
    return this_var

In [3]:
flattened_test_disam_list = reload_vars('data/vars/flattened_test_disam_list.pkl')
flattened_train_disamb_list = reload_vars('data/vars/flattened_train_disamb_list.pkl')
flattened_disam_list = flattened_test_disam_list + flattened_train_disamb_list
df_flattened_disam_list = pd.DataFrame(flattened_disam_list)
df_flattened_disam_list

,title,term,definition
0,Definition:Boundary (Geometry),Boundary,"\n\nFor example, the endpoints of a line segm..."
1,Definition:Boundary (Topology),Boundary,"Let T = ( S, τ) be a topological space.\n\nLet..."
2,Definition:Boundary (Graph Theory),Boundary,"=== Simple Graph ===\nLet G = ( V, E ) be a si..."
3,Definition:Right (Direction),Right,The direction right is that way:\n:→
4,Definition:Right Angle,Right,A right angle is an angle that is equal to hal...
...,...,...,...
1864,Definition:Congruence (Metric Spaces),Congruence,"Let ( X, d ) be a metric space.\n\nTwo subsets..."
1865,Definition:Congruence Modulo an Ideal,Congruence,"Let ( R, +, ∘) be a ring, and let J be an idea..."
1866,Definition:Congruence Relation,Congruence,"Let ( S, ∘) be an algebraic structure.\n\nLet ..."
1867,Definition:Congruence (Number Theory),Congruence,Let z ∈ℝ.\n\n\n=== Definition by Remainder aft...


In [19]:
%%time
model_path_list = ['sentence-transformers/all-MiniLM-L12-v2',
             'sentence-transformers/all-MiniLM-L6-v2',
             'InriaValda/cc_math_bert_ep10',# pretrained from scratch with MLM
             'InriaValda/cc_math_roberta_ep10', # pretrained from scratch with MLM
             'bert-base-uncased',
             'math-similarity/Bert-MLM_arXiv',
             'math-similarity/Bert-MLM_arXiv-MP-class_zbMath',
             'Lihuchen/pearl_small'
             ]
for p in model_path_list:
    tokenizer = AutoTokenizer.from_pretrained(p)
    df_flattened_disam_list[f"{p.split('/')[-1]}_tokenized_title_len"] = tokenizer(list(df_flattened_disam_list['title']), return_length=True).length
    df_flattened_disam_list[f"{p.split('/')[-1]}_tokenized_term_len"] = tokenizer(list(df_flattened_disam_list['term']), return_length=True).length
    df_flattened_disam_list[f"{p.split('/')[-1]}_tokenized_definition_len"] = tokenizer(list(df_flattened_disam_list['definition']), return_length=True).length

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 29.4 s, sys: 632 ms, total: 30 s
Wall time: 3.5 s


In [20]:
df_flattened_disam_list.describe()

,all-MiniLM-L12-v2_tokenized_title_len,all-MiniLM-L12-v2_tokenized_term_len,all-MiniLM-L12-v2_tokenized_definition_len,all-MiniLM-L6-v2_tokenized_title_len,all-MiniLM-L6-v2_tokenized_term_len,all-MiniLM-L6-v2_tokenized_definition_len,cc_math_bert_ep10_tokenized_title_len,cc_math_bert_ep10_tokenized_term_len,cc_math_bert_ep10_tokenized_definition_len,cc_math_roberta_ep10_tokenized_title_len,...,bert-base-uncased_tokenized_definition_len,Bert-MLM_arXiv_tokenized_title_len,Bert-MLM_arXiv_tokenized_term_len,Bert-MLM_arXiv_tokenized_definition_len,Bert-MLM_arXiv-MP-class_zbMath_tokenized_title_len,Bert-MLM_arXiv-MP-class_zbMath_tokenized_term_len,Bert-MLM_arXiv-MP-class_zbMath_tokenized_definition_len,pearl_small_tokenized_title_len,pearl_small_tokenized_term_len,pearl_small_tokenized_definition_len
count,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,...,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000,1869.000000
mean,8.520064,3.360621,138.789192,8.520064,3.360621,138.789192,7.888175,3.098983,134.768325,8.884430,...,138.789192,8.520064,3.360621,138.789192,8.520064,3.360621,138.789192,8.520064,3.360621,138.789192
std,2.191146,0.772570,225.788354,2.191146,0.772570,225.788354,1.885313,0.349891,220.630905,2.288721,...,225.788354,2.191146,0.772570,225.788354,2.191146,0.772570,225.788354,2.191146,0.772570,225.788354
min,5.000000,3.000000,2.000000,5.000000,3.000000,2.000000,5.000000,3.000000,2.000000,5.000000,...,2.000000,5.000000,3.000000,2.000000,5.000000,3.000000,2.000000,5.000000,3.000000,2.000000
25%,7.000000,3.000000,47.000000,7.000000,3.000000,47.000000,6.000000,3.000000,46.000000,7.000000,...,47.000000,7.000000,3.000000,47.000000,7.000000,3.000000,47.000000,7.000000,3.000000,47.000000
50%,8.000000,3.000000,81.000000,8.000000,3.000000,81.000000,8.000000,3.000000,79.000000,9.000000,...,81.000000,8.000000,3.000000,81.000000,8.000000,3.000000,81.000000,8.000000,3.000000,81.000000
75%,10.000000,3.000000,149.000000,10.000000,3.000000,149.000000,9.000000,3.000000,145.000000,10.000000,...,149.000000,10.000000,3.000000,149.000000,10.000000,3.000000,149.000000,10.000000,3.000000,149.000000
max,19.000000,7.000000,5416.000000,19.000000,7.000000,5416.000000,18.000000,7.000000,5348.000000,22.000000,...,5416.000000,19.000000,7.000000,5416.000000,19.000000,7.000000,5416.000000,19.000000,7.000000,5416.000000


In [17]:
df_flattened_disam_list.describe().to_csv("comp_tokenized_len.csv")

In [4]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
tokenizer('Definition:Congruence Relation')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'input_ids': [101, 6210, 1024, 26478, 6820, 10127, 7189, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
tokenizer(['Definition:Congruence Relation', 'Definition:Right (Direction)'],return_length=True)

{'input_ids': [[101, 6210, 1024, 26478, 6820, 10127, 7189, 102], [101, 6210, 1024, 2157, 1006, 3257, 1007, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'length': [8, 8]}

In [10]:
tokenizer.convert_ids_to_tokens([101, 6210, 1024, 26478, 6820, 10127, 7189, 102], skip_special_tokens=True)

['definition', ':', 'cong', '##ru', '##ence', 'relation']